# グランディ画像生成コード

## ニムの仕様
- 山の数は二つ
- 山から石をとるときの取り方は，両方の山から0個以上同時にとる
    - ただし，とった石の数の合計は1個以上
- 遷移集合（とり方の種類）には1つ以上の集合が含まれる

## コード

### 大きな流れ
1. 遷移集合のリストを作成
2. その時のグランディ数テーブル配列を作成
3. 配列を画像として保存

### インポート

In [73]:
#import seaborn as sns
#import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
#from IPython import display
from PIL import Image
from scipy import signal
import itertools

### 定数

#### 定数の説明

In [74]:
OUTPUT_DIR_PATH: str #画像を出力するディレクトリのパス

N_SIZE: int #山1の石の総数
M_SIZE: int #山2の石の総数
ALL_MOVE_RESTRICTIONS: list #すべての遷移集合，( 例: [(n1,m1),(n2,n3),・・・] )
GRUNDY_TABLE: np.array #グランディ数のテーブル
GRUNDYNUM_MAX: int  #グランディ数の最大値

MIN_NUM_OF_MOVE_RESTRICTIONS: int #遷移集合の数の最小値
MAX_NUM_OF_MOVE_RESTRICTIONS: int #遷移集合の数の最大値
MIN_NUM_OF_MOVE: int #遷移する最小の石の個数
MAX_NUM_OF_MOVE: int #遷移する最大の石の個数

#### 定数の設定

In [75]:
OUTPUT_DIR_PATH = "./output/" #画像を出力するディレクトリのパス

N_SIZE = 256 #石の個数n
M_SIZE = 256 #石の個数m

MIN_NUM_OF_MOVE = 0 #取ることのできる最小の石の個数
MAX_NUM_OF_MOVE = 9 #取ることのできる最大の石の個数
# もしMIN_NUM_OF_MOVE=0, MAX_NUM_OF_MOVE=2なら
# 使える移動は [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)] である

MIN_NUM_OF_MOVE_RESTRICTIONS = 4 #移動条件数の最小値
MAX_NUM_OF_MOVE_RESTRICTIONS = 4 #移動条件数の最大値

### 関数

#### すべての遷移集合を含んだリストを生成

In [76]:
def makeAllMoveRestrictions():
    #--- すべての遷移集合の生成 ---
    # intertools.productを使って，最小値と最大値の間の数の組み合わせを生成
    res = list(itertools.product(range(MIN_NUM_OF_MOVE, MAX_NUM_OF_MOVE + 1), repeat=2)) 

    # 条件より(0,0)を除く
    if (0, 0) in res:
        res.remove((0, 0))

    print("--- すべての遷移集合の生成 makeAllMoveRestrictions() ---")
    # print("取ることのできる最小の石の個数は", MIN_NUM_OF_MOVE, "~", MAX_NUM_OF_MOVE, "です")
    # print("移動条件数は", MIN_NUM_OF_MOVE_RESTRICTIONS, "~", MAX_NUM_OF_MOVE_RESTRICTIONS, "です")
    print("すべての遷移集合 :", res, "\n")
    return res

ALL_MOVE_RESTRICTIONS = makeAllMoveRestrictions()

--- すべての遷移集合の生成 makeAllMoveRestrictions() ---
すべての遷移集合 : [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)] 



#### 条件に沿った遷移集合の生成

In [77]:

def makeMoveRestrictions():
   print("--- 遷移集合の生成 makeMoveRestrictions() ---")
   for num_of_move_restrictions in range(MIN_NUM_OF_MOVE_RESTRICTIONS, MAX_NUM_OF_MOVE_RESTRICTIONS + 1):
      move_restrictions_list = list(itertools.combinations(ALL_MOVE_RESTRICTIONS, num_of_move_restrictions)) #コンビネーション
      #move_restrictions_list = list(itertools.product(li, repeat=num_of_move_restrictions)) #重複ありコンビネーション
      print("遷移集合の要素数が", num_of_move_restrictions, "の時, 生成される画像は", len(move_restrictions_list), "個")
   return move_restrictions_list

MOVE_RESTRICTIONS = makeMoveRestrictions()[0]
print("遷移集合 :", MOVE_RESTRICTIONS, "\n")


--- 遷移集合の生成 makeMoveRestrictions() ---
遷移集合の要素数が 4 の時, 生成される画像は 3764376 個
遷移集合 : ((0, 1), (0, 2), (0, 3), (0, 4)) 



#### グランディテーブルの生成

##### リストのうち最小非負整数を返す

In [78]:
# mex関数 数列liに含まれない最小非負整数を返す
def mex(li):
    return min( set([i for i in range(0, max(li+[-1])+3)])-set(li) )

##### グランディテーブルの生成

In [79]:
# (N,M)の状態の二山ニムのグランディ数を求める
# (N,M)のグランディ数を求める過程で全てのグランディ数が求まる
def fillGrundyTable():
    global GRUNDY_TABLE
    global GRUNDYNUM_MAX

    GRUNDY_TABLE = np.full((N_SIZE, M_SIZE), -1) #初期化
    GRUNDYNUM_MAX = -1
    
    print("--- グランディ数のテーブルの生成 fillGrundyTable() ---")

    for n in range(N_SIZE): # nは山1の石の数
        for m in range(M_SIZE): # mは山2の石の数
            next_GrundyNum_list = [] # 次の状態のグランディ数のリスト
            for move in MOVE_RESTRICTIONS: # 遷移集合の各要素について
                next_stone_num = np.array([n, m]) - np.array([move[0], move[1]]) # 次の状態の石の数
                if 0 <= next_stone_num[0] and 0 <= next_stone_num[1]: # 両方が0以上なら
                    next_GrundyNum_list.append(GRUNDY_TABLE[next_stone_num[0]][next_stone_num[1]]) # グランディテーブルからグランディ数を取得
            GRUNDY_TABLE[n][m] = mex(next_GrundyNum_list) # mex関数を用いてグランディ数を求めて代入

            if GRUNDYNUM_MAX < GRUNDY_TABLE[n][m]: # グランディ数の最大値を更新
                GRUNDYNUM_MAX = GRUNDY_TABLE[n][m]
            print("n:", n, "m:", m, ",", GRUNDY_TABLE[n][m])
fillGrundyTable()

--- グランディ数のテーブルの生成 fillGrundyTable() ---
n: 0 m: 0 , 0
n: 0 m: 1 , 1
n: 0 m: 2 , 2
n: 0 m: 3 , 3
n: 0 m: 4 , 4
n: 0 m: 5 , 0
n: 0 m: 6 , 1
n: 0 m: 7 , 2
n: 0 m: 8 , 3
n: 0 m: 9 , 4
n: 0 m: 10 , 0
n: 0 m: 11 , 1
n: 0 m: 12 , 2
n: 0 m: 13 , 3
n: 0 m: 14 , 4
n: 0 m: 15 , 0
n: 0 m: 16 , 1
n: 0 m: 17 , 2
n: 0 m: 18 , 3
n: 0 m: 19 , 4
n: 0 m: 20 , 0
n: 0 m: 21 , 1
n: 0 m: 22 , 2
n: 0 m: 23 , 3
n: 0 m: 24 , 4
n: 0 m: 25 , 0
n: 0 m: 26 , 1
n: 0 m: 27 , 2
n: 0 m: 28 , 3
n: 0 m: 29 , 4
n: 0 m: 30 , 0
n: 0 m: 31 , 1
n: 0 m: 32 , 2
n: 0 m: 33 , 3
n: 0 m: 34 , 4
n: 0 m: 35 , 0
n: 0 m: 36 , 1
n: 0 m: 37 , 2
n: 0 m: 38 , 3
n: 0 m: 39 , 4
n: 0 m: 40 , 0
n: 0 m: 41 , 1
n: 0 m: 42 , 2
n: 0 m: 43 , 3
n: 0 m: 44 , 4
n: 0 m: 45 , 0
n: 0 m: 46 , 1
n: 0 m: 47 , 2
n: 0 m: 48 , 3
n: 0 m: 49 , 4
n: 0 m: 50 , 0
n: 0 m: 51 , 1
n: 0 m: 52 , 2
n: 0 m: 53 , 3
n: 0 m: 54 , 4
n: 0 m: 55 , 0
n: 0 m: 56 , 1
n: 0 m: 57 , 2
n: 0 m: 58 , 3
n: 0 m: 59 , 4
n: 0 m: 60 , 0
n: 0 m: 61 , 1
n: 0 m: 62 , 2
n: 0 m: 63 , 3
n: 0 m: 6

#### グランディテーブルの表示

In [80]:
def showGrandyTable():
    print("--- グランディ数のテーブルの表示 showGrandyTable() ---")
    print("グランディ数の最大値は", GRUNDYNUM_MAX, "です")
    for row in GRUNDY_TABLE:
        for col in row:
            print(col, end=" ")
        print()

showGrandyTable()

--- グランディ数のテーブルの表示 showGrandyTable() ---
グランディ数の最大値は 4 です
0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 
0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4 0 1 2 3 4

#### グランディ画像の生成

##### 平均化フィルタ

In [81]:
# 平均化フィルタを畳み込み
def myConvolve2d(array, n=3):
    if n < 3:
        n = 3
        print("Warning in func:myConvolve2d, arg n is too small")
    filter = [[1/(n*n) for a in range(n)] for b in range(n)]
    filter = np.array([np.array(row) for row in filter])
    array = signal.convolve2d(array, filter, mode="same", boundary="wrap")
    return array

#### グランディ画像の生成

In [83]:
def saveGTable2Image(ave_filter=False, filter_size=3, show_output_file_path=True, return_image_array=False, image_fmt="bmp"):
    global GRUNDY_TABLE
    global MOVE_RESTRICTIONS
    global GRUNDYNUM_MAX

    Grundy_table = GRUNDY_TABLE
    move_restrictions = MOVE_RESTRICTIONS
    GurundyNum_max = GRUNDYNUM_MAX

    # 0 ~ 255の範囲で正規化
    if GurundyNum_max == 0:
        output_table = Grundy_table
    else:
        output_table = Grundy_table * (255 / GurundyNum_max)
    # 出力画像用に白黒反転する（つまりグランディ数が0の場所を白色にしている）
    output_table = 255 - output_table

    filter_name = ""
    # もし画像をぼかすするなら以下を実行
    if ave_filter == True:
        filter_name += "_" + "ave_filter"
        output_table = myConvolve2d(output_table,filter_size)
    
    # uint8にキャストして出力イメージ用フォーマットにする
    output_img = Image.fromarray(output_table.astype(np.uint8))

    # ファイルパスとファイル名を作成
    file_name = ""
    for move in move_restrictions:
        file_name += str(move[0]) + str( move[1]) + "_"
        #file_name += str(move[0]) + str( move[1]) + "_"
    file_name += filter_name

    full_file_path = OUTPUT_DIR_PATH + file_name + "." + image_fmt #最終的なファイル名

    if show_output_file_path:
        print(full_file_path)

    output_img.save(full_file_path)
    

    if return_image_array:
        return output_table
    
saveGTable2Image(ave_filter=False, filter_size=3, show_output_file_path=True, return_image_array=False, image_fmt="bmp")

./output/01_02_03_04_.bmp
